In [13]:
import requests
from datetime import datetime
from dotenv import load_dotenv
import os

load_dotenv()

# Get Strava Access Token
access_token = os.getenv("STRAVA_ACCESS_TOKEN")

response = requests.get(
    "https://www.strava.com/api/v3/athlete/activities",
    headers={"Authorization": f"Bearer {access_token}"}
)

#print(response.json())

# Assuming `activities` is your list of activity JSON from Strava
def display_summary(activities):
    for activity in activities:
        name = activity.get("name")
        sport_type = activity.get("sport_type")
        distance_km = round(activity.get("distance", 0) / 1000, 2)
        time_min = round(activity.get("moving_time", 0) / 60, 1)
        date = datetime.strptime(activity["start_date"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
        heart = f"{activity['average_heartrate']} bpm" if activity.get("average_heartrate") else "n/a"

        print(f"🏋️ {name} | {sport_type} | {distance_km} km | {time_min} min | {date} | HR: {heart}")

# Example usage
display_summary(response.json())  # if you have raw response

🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 48.5 min | 2025-04-24 | HR: n/a
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 111.5 min | 2025-04-23 | HR: n/a
🏋️ Evening Run | Run | 5.23 km | 30.1 min | 2025-04-22 | HR: 185.4 bpm
🏋️ Evening workout 🏋️ | WeightTraining | 0.0 km | 94.6 min | 2025-04-21 | HR: n/a
🏋️ Morning Run | Run | 5.1 km | 29.3 min | 2025-04-21 | HR: 183.4 bpm
🏋️ Morning workout ☀️ | WeightTraining | 0.0 km | 68.6 min | 2025-04-19 | HR: n/a
🏋️ Evening workout 🏋️ | WeightTraining | 0.0 km | 79.0 min | 2025-04-18 | HR: n/a
🏋️ Lunch Run | Run | 4.04 km | 22.2 min | 2025-04-17 | HR: 190.4 bpm
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 108.2 min | 2025-04-14 | HR: n/a
🏋️ Lunch Run | Run | 5.1 km | 29.2 min | 2025-04-13 | HR: 190.7 bpm
🏋️ Morning workout ☀️ | WeightTraining | 0.0 km | 91.2 min | 2025-04-12 | HR: n/a
🏋️ Afternoon workout 💪 | WeightTraining | 0.0 km | 101.2 min | 2025-04-11 | HR: n/a
🏋️ Paris Run🥖🇫🇷 | Run | 5.06 km | 29.4 min | 2025-04-05 | H

In [12]:
import json

with open("strava_activities.json", "w") as f:
    json.dump(response.json(), f, indent=4)

print("Exported to strava_activities.json")

Exported to strava_activities.json


In [13]:
import csv
from datetime import datetime

activities = response.json()

with open("strava_activities_detailed.csv", "w", newline="") as csvfile:
    fieldnames = [
        "name", "sport_type", "distance_km", "moving_time_min", "elapsed_time_min",
        "elevation_gain_m", "average_heartrate", "max_heartrate", "average_speed_kmh",
        "start_date", "start_latlng", "end_latlng", "kudos_count", "suffer_score"
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for activity in activities:
        writer.writerow({
            "name": activity.get("name"),
            "sport_type": activity.get("sport_type"),
            "distance_km": round(activity.get("distance", 0) / 1000, 2),
            "moving_time_min": round(activity.get("moving_time", 0) / 60, 1),
            "elapsed_time_min": round(activity.get("elapsed_time", 0) / 60, 1),
            "elevation_gain_m": activity.get("total_elevation_gain", 0),
            "average_heartrate": activity.get("average_heartrate", "n/a"),
            "max_heartrate": activity.get("max_heartrate", "n/a"),
            "average_speed_kmh": round(activity.get("average_speed", 0) * 3.6, 2),
            "start_date": datetime.strptime(activity["start_date"], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d"),
            "start_latlng": activity.get("start_latlng", []),
            "end_latlng": activity.get("end_latlng", []),
            "kudos_count": activity.get("kudos_count", 0),
            "suffer_score": activity.get("suffer_score", "n/a"),
        })

print("Exported to strava_activities_detailed.csv")

Exported to strava_activities_detailed.csv


In [11]:
import os
from openai import OpenAI

load_dotenv()

# Set up the OpenAI client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# Load your activities
with open("strava_activities.json") as f:
    activities = json.load(f)

# Summarize last 10 workouts
summary = "\n".join([
    f"{a['name']} - {a['sport_type']} - {round(a['distance'] / 1000, 2)} km on {a['start_date'][:10]}"
    for a in activities[:10]
])

# Build prompt
prompt = f"""
Here are my last 10 workouts from Strava:
{summary}

My height is 180 cm and weight is 80 kg.

- Calculate my daily calorie requirement.
- Suggest a balanced dietary plan (breakfast, lunch, dinner, snacks) for the whole day.
- Give calories and macros (protein, carbs, fats in grams) for each meal.
- Add a simple recipe for each meal.
"""

# Now create chat completion using the client
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

To calculate your daily calorie requirement, we need to consider your Basal Metabolic Rate (BMR) and physical activity level. Given your height (180 cm) and weight (80 kg), we'll use the Mifflin-St Jeor equation to estimate your BMR. We'll also assume you're male and have a moderate physical activity level based on your workout routine:

### Step 1: Calculate BMR
\[ \text{BMR} = 10 \times \text{weight in kg} + 6.25 \times \text{height in cm} - 5 \times \text{age} + 5 \]

Assuming you're 30 years old (generally typical for simple calculations):
\[ \text{BMR} = 10 \times 80 + 6.25 \times 180 - 5 \times 30 + 5 \]
\[ \text{BMR} = 800 + 1125 - 150 + 5 \]
\[ \text{BMR} = 1780 \, \text{calories/day} \]

### Step 2: Estimate Total Daily Energy Expenditure (TDEE)
Assuming moderate activity (1.55 factor):
\[ \text{TDEE} = \text{BMR} \times \text{Activity Level} \]
\[ \text{TDEE} = 1780 \times 1.55 \]
\[ \text{TDEE} = 2759 \, \text{calories/day} \]

Based on this, your daily calorie requirement i

ID: asst_FYRCP0guynI7TiZ6VXBUyAbK | Name: None
